<center>
    <p style="text-align:center">
        <img alt="phoenix logo" src="https://storage.googleapis.com/arize-assets/phoenix/assets/phoenix-logo-light.svg" width="200"/>
        <br>
        <a href="https://docs.arize.com/phoenix/">Docs</a>
        |
        <a href="https://github.com/Arize-ai/phoenix">GitHub</a>
        |
        <a href="https://join.slack.com/t/arize-ai/shared_invite/zt-1px8dcmlf-fmThhDFD_V_48oU7ALan4Q">Community</a>
    </p>
</center>
<h1 align="center">Tracing and Evaluating a LangChain OpenAI Agent Application</h1>

With the new OpenAI API that supports function calling, it’s never been easier to build your own agent.

In this notebook tutorial, we showcase how to write your own OpenAI agent in under 50 lines of code and use Phoenix to inspect the internals of the Agent. It is minimal, yet feature complete (with ability to carry on a conversation and use tools).

## 1. Install Dependencies and Import Libraries

Install Phoenix, LangChain, and OpenAI.

In [ ]:
!pip install -qq arize-phoenix langchain openai

Import libraries.

In [ ]:
import os
from getpass import getpass

import openai
import phoenix as px
from langchain.agents import AgentType, Tool, initialize_agent
from langchain.chains import LLMMathChain
from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate, MessagesPlaceholder
from phoenix.trace.langchain import LangChainInstrumentor

## 2. Launch Phoenix

You can run Phoenix in the background to collect trace data emitted by any LangChain application that has been instrumented with the `OpenInferenceTracer`.

Launch Phoenix and follow the instructions in the cell output to open the Phoenix UI (the UI should be empty because we have yet to run a LangChain application).

In [ ]:
session = px.launch_app()

## 3. Configure Your OpenAI Credentials

Let's make sure we have openAI credentials set up.

In [ ]:
if not (openai_api_key := os.getenv("OPENAI_API_KEY")):
    openai_api_key = getpass("🔑 Enter your OpenAI API key: ")
openai.api_key = openai_api_key
os.environ["OPENAI_API_KEY"] = openai_api_key

## 4. Build and Instrument Your Agent

Let's now instrument LangChain to send trace data to Phoenix. Since we want all parts of LangChain to be traced, we will use the `LangChainInstrumentor` to instrument the entire application with an `OpenInferenceTracer`. For more fine-grained control, you can use the `OpenInferenceTracer` on sub-parts of your application. For the full details, please consult the [LangChain documentation](https://docs.arize.com/phoenix/integrations/langchain).

In [ ]:
LangChainInstrumentor().instrument()

Let's now define the LLM model we will use for our agent.

In [ ]:
llm = ChatOpenAI(temperature=0, model="gpt-3.5-turbo-0613")

Let's define the tools the LLM will have at its disposal. We will use the following tools:

In [ ]:
llm_math_chain = LLMMathChain.from_llm(llm=llm, verbose=True)
# Let's give the LLM access to math tools
tools = [
    Tool(
        name="Calculator",
        func=llm_math_chain.run,
        description="useful for when you need to answer questions about math",
    ),
]

Let's setup the Prompt Template. This will inform how the agent will respond to queries.

In [ ]:
prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a helpful assistant"),
        ("human", "{input}"),
        MessagesPlaceholder(variable_name="agent_scratchpad"),
    ]
)

Now, we define our agent that’s capable answering questions and calling tools.

The meat of the agent logic breaks down into three steps:

- Call OpenAI to decide which tool (if any) to call and with what arguments.

- Call the tool with the arguments to obtain an output

- Call OpenAI to synthesize a response from the conversation context and the tool output.

Let's initialize the agent.

In [ ]:
llm = ChatOpenAI(temperature=0, model="gpt-3.5-turbo-0613")
agent_executor = initialize_agent(tools, llm, agent=AgentType.OPENAI_FUNCTIONS, verbose=True)

## 5. Chat With Your Agent

Let's now chat with our agent! Note that the `OpenInferenceTracer` is logging traces of the execution to phoenix as you invoke the agent!

In [ ]:
response = agent_executor.invoke({"input": "What is 47 raised to the 5th power?"})
response

Let's chat with our agent a few more times. This time with some follow-up questions.

In [ ]:
queries = [
    "What is (121 * 3) + 42?",
    "what is 3 * 3?",
    "what is 4 * 4?",
    "what is 75 * (3 + 4)?",
    "what is 23 times 87",
]

for query in queries:
    print(f"> {query}")
    response = agent_executor.invoke({"input": query})
    print(response)
    print("---")

Open the `session.url` in your browser to take a look at the traces in Phoenix. Note that LLM spans contain the OpenAI function calls, and that we can inspect what tool the LLM picked based on the queries.

To learn more about function calling, check out the [OpenAI API docs](https://openai.com/blog/function-calling-and-other-api-updates).


In [ ]:
print(f"View the traces in phoenix: {session.url}")

## 6. Export Your Trace Data

You can export your trace data as a pandas dataframe for further analysis and evaluation.

In [ ]:
trace_df = session.get_spans_dataframe()
trace_df